<a href="https://colab.research.google.com/github/ayushi-2310/AI_Agent/blob/main/MarketResearch_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
# !pip install -q fastapi nest-asyncio uvicorn pandas openai google-generativeai colabcode

In [109]:
import os
import re
import requests
import pandas as pd
from fastapi import FastAPI, Form, UploadFile, File
from fastapi.responses import HTMLResponse
from pyngrok import ngrok
import nest_asyncio, uvicorn
from typing import List

In [110]:
# --- Config ---
from google.colab import userdata

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", userdata.get("GOOGLE_API_KEY"))
GEMINI_MODEL = "gemini-1.5-flash"
GEMINI_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{GEMINI_MODEL}:generateContent?key={GEMINI_API_KEY}"
app = FastAPI()

In [111]:
knowledge_graph = {
    "CTR": {"impacts": ["Conversions"], "influenced_by": ["Ad Text", "Targeting"]},
    "CPC": {"impacts": ["ROI"], "influenced_by": ["Bidding Strategy"]},
}


In [112]:
def call_gemini(prompt):
    payload = {
        "contents": [{"parts": [{"text": prompt}]}],
        "generationConfig": {
            "maxOutputTokens": 256,
            "temperature": 0.7,
            "topP": 0.9,
            "topK": 40
        }
    }
    resp = requests.post(GEMINI_URL, json=payload)
    if resp.status_code != 200:
        return f"[ERROR] {resp.status_code}: {resp.text}"
    parts = resp.json().get("candidates", [{}])[0].get("content", {}).get("parts", [])
    return parts[0].get("text", "") if parts else "[No response]"


In [113]:
@app.get("/", response_class=HTMLResponse)
async def get_home():
    return """
    <html><head><title>Ad Copy Rewriter</title></head>
    <body style="font-family:Arial; padding:20px;">
        <h2>📝 Rewrite Your Ad Copy</h2>
        <form action="/generate" method="post">
            <textarea name="content" rows="6" cols="70" placeholder="Enter your original ad copy..." required></textarea><br><br>
            <label><strong>Select Tone:</strong></label>
            <select name="tone">
                <option value="fun">Fun</option>
                <option value="professional">Professional</option>
                <option value="bold">Bold</option>
                <option value="conversational">Conversational</option>
                <option value="urgent">Urgent</option>
            </select><br><br>
            <button type="submit">Generate</button>
        </form>
        <hr>
        <h3>📂 Upload CSV for Campaign Analysis</h3>
        <form action="/upload" enctype="multipart/form-data" method="post">
            <input name="file" type="file" accept=".csv">
            <button type="submit">Upload & Analyze</button>
        </form>
    </body></html>
    """

In [114]:
@app.post("/generate", response_class=HTMLResponse)
async def generate_content(content: str = Form(...), tone: str = Form(...)):
    prompt = f"Rewrite the following ad copy in a {tone} tone, optimized for high engagement:\n\n{content}"
    generated_text = call_gemini(prompt)

    # Evaluation
    response_length = len(generated_text.split())
    tone_keywords = {
        "fun": ["exciting", "wow", "fun", "awesome", "playful", "lol", "party"],
        "professional": ["professional", "trusted", "experience", "solutions", "partner"],
        "bold": ["guaranteed", "best ever", "never seen before", "unbeatable"],
        "conversational": ["you", "let’s", "imagine", "we’ve got"],
        "urgent": ["now", "limited", "don’t wait", "only", "act fast"]
    }
    matched_keywords = [kw for kw in tone_keywords.get(tone, []) if kw in generated_text.lower()]
    tone_match_score = round(len(matched_keywords) / max(len(tone_keywords[tone]), 1), 2)
    hallucination_flag = "?" if "aliens" in generated_text.lower() else "No obvious hallucinations"

    html_output = f"""
    <html><head><style>body{{font-family:Arial;padding:20px;}}</style></head>
    <body>
        <h2>🎯 Rewritten Ad Copy ({tone.title()} Tone)</h2>
        <p><strong>Original:</strong> {content}</p><hr>
        <pre>{generated_text}</pre>
        <h3>📊 Evaluation Metrics</h3>
        <ul>
            <li><strong>Word Count:</strong> {response_length}</li>
            <li><strong>Tone Match Score:</strong> {tone_match_score*100}%</li>
            <li><strong>Hallucination Check:</strong> {hallucination_flag}</li>
        </ul>
        <h3>🔁 Feedback</h3>
        <form action="/feedback" method="post">
            <input type="hidden" name="prompt" value="{content}">
            <input type="hidden" name="output" value="{generated_text}">
            <label>Was this helpful?</label>
            <select name="feedback">
                <option value="positive">👍 Yes</option>
                <option value="negative">👎 No</option>
            </select>
            <button type="submit">Submit</button>
        </form>
        <br><a href="/">🔙 Back</a>
    </body></html>
    """
    return HTMLResponse(content=html_output)

In [115]:
@app.post("/feedback")
async def feedback(prompt: str = Form(...), output: str = Form(...), feedback: str = Form(...)):
    with open("feedback_log.csv", "a") as f:
        f.write(f'"{prompt}","{output}","{feedback}"\n')
    return HTMLResponse("<h2>✅ Thank you for your feedback!</h2><a href='/'>🔙 Back</a>")


In [116]:
@app.post("/upload")
async def upload_csv(file: UploadFile = File(...)):
    df = pd.read_csv(file.file)
    summary_prompt = f"Here’s a CSV summary:\n\n{df.describe().to_string()}\n\n"
    step_1 = call_gemini("Summarize ad campaign performance:\n" + summary_prompt)
    step_2 = call_gemini("Given the summary, what metrics are underperforming?\n" + step_1)
    step_3 = call_gemini("Suggest improvements to boost the underperforming metrics:\n" + step_2)
    step_4 = call_gemini("Generate a rewritten ad copy to reflect those suggestions:\n" + step_3)

    return HTMLResponse(f"""
        <html><body style="font-family:Arial; padding:20px;">
        <h2>📈 Agentic Analysis</h2>
        <h3>Step 1: Summary</h3><pre>{step_1}</pre>
        <h3>Step 2: Issues</h3><pre>{step_2}</pre>
        <h3>Step 3: Improvements</h3><pre>{step_3}</pre>
        <h3>Step 4: Rewritten Ad</h3><pre>{step_4}</pre>
        <a href="/">🔙 Back</a></body></html>
    """)

In [117]:
@app.post("/evaluate")
async def evaluate_ad(output: str = Form(...), expected_keywords: str = Form(...)):
    expected = expected_keywords.lower().split(",")
    match_score = sum(1 for word in expected if word.strip() in output.lower()) / max(len(expected), 1)
    return {"keyword_match_score": round(match_score, 2), "output_length": len(output.split())}

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-35' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

In [118]:
memory_log = []
@app.post("/log-feedback")
async def log_feedback(prompt: str = Form(...), output: str = Form(...), feedback: str = Form(...)):
    memory_log.append({"prompt": prompt, "output": output, "feedback": feedback})
    return {"status": "logged", "log_size": len(memory_log)}

In [119]:
# from pyngrok import ngrok
# from google.colab import userdata
# import os

# # Set your ngrok authtoken from Colab secrets
# NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
# if NGROK_AUTH_TOKEN:
#     ngrok.set_auth_token(NGROK_AUTH_TOKEN)
#     print("ngrok authtoken set successfully.")
# else:
#     print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")

In [120]:
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

# Enable Colab support
nest_asyncio.apply()

# Run FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)


Public URL: https://1ef0-34-168-140-187.ngrok-free.app


INFO:     Started server process [288]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2401:4900:883e:2e95:8996:960:4906:c515:0 - "GET / HTTP/1.1" 200 OK
INFO:     2401:4900:883e:2e95:8996:960:4906:c515:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2401:4900:883e:2e95:8996:960:4906:c515:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [288]
